In [ ]:
import os
os.chdir("/opt/project")

import json
import psutil
import numpy as np
from settings import TEST_DATA_PATH

import keras
keras.mixed_precision.set_global_policy("mixed_float16")

from figures import ModelFigureGenerator
from datasets import ECGSequence, get_files
from model import ModelSpesification, get_model

In [ ]:
model_id = 0
model_spec = ModelSpesification(model_id, tags=("best",))

In [ ]:
# Model settings
lr = 1e-3
batch_size = 64
val_split = 0.02
shuffle_on_epoch_end = True
worker_num = psutil.cpu_count(logical=True) - 2

opt = keras.optimizers.Adam(lr)
loss = keras.losses.BinaryCrossentropy()

# Callback settings
stop_patience = 9
reduce_patience = 7
stop_min_delta = 1e-5
reduce_min_lr = lr * 1e-2

callbacks = [
    # Learning Optimizers
    keras.callbacks.EarlyStopping(patience=stop_patience, min_delta=stop_min_delta),
    keras.callbacks.ReduceLROnPlateau(patience=reduce_patience, min_lr=reduce_min_lr),
    # Logs
    keras.callbacks.TensorBoard(log_dir=model_spec.log_dir, write_graph=False),
    keras.callbacks.CSVLogger(model_spec.log_dir / "training.log", append=False),
    # Checkpoints
    keras.callbacks.ModelCheckpoint(model_spec["best"].model_dir / "model.keras",
                                    save_best_only=True),
]

train_seq, valid_seq = ECGSequence.get_train_and_val(
    get_files("input"), get_files("label"), "tracings",
    batch_size, val_split, drop=1, shuffle=shuffle_on_epoch_end,
    workers=worker_num, use_multiprocessing=True
)

# If you are continuing an interrupted section, uncomment the line below:
# model = keras.models.load_model(model_spec.model_dir / "model.keras", compile=False)
model = get_model(train_seq.n_classes)
model.compile(loss=loss, optimizer=opt)

In [ ]:
# Train neural network
with open(model_spec.model_dir / "params.json", "w") as file:
    json.dump({
        "lr": lr, "opt": opt.name, "loss": loss.name, "val_split": val_split,
        "batch_size": batch_size, "stop_patience": stop_patience,
        "reduce_patience": reduce_patience, "stop_min_delta": stop_min_delta,
        "reduce_min_lr": reduce_min_lr, "shuffle_on_epoch_end": shuffle_on_epoch_end,
    }, file, indent=4)

# If you are continuing an interrupted section change initial epoch
history = model.fit(train_seq,
                    epochs=70,
                    initial_epoch=0,
                    callbacks=callbacks,
                    validation_data=valid_seq,
                    verbose=1)
# Save final result
model.save(model_spec.model_dir / "model.keras")

In [ ]:
# Evaluate the model on the test set
seq = ECGSequence([TEST_DATA_PATH / "ecg_tracings.hdf5"], None, "tracings", batch_size=1)
y_score = model.predict(seq, verbose=1)

np.save(model_spec.model_dir / "prediction.npy", y_score)

In [ ]:
# Generate figures and tables
stats = ModelFigureGenerator(model_spec)

stats.generate_table_two()
stats.generate_supplementary_table_one()
stats.generate_supplementary_figure_one()

In [ ]:
# Inspect training logs
%load_ext tensorboard
%tensorboard --logdir {model_spec.log_dir} --bind_all